In [ ]:
import sys, os

In [ ]:
from exllamav2 import ExLlamaV2, ExLlamaV2Config, ExLlamaV2Cache, ExLlamaV2Tokenizer
from exllamav2.generator import ExLlamaV2StreamingGenerator, ExLlamaV2Sampler

In [ ]:
total_cache_tokens = 16384

In [ ]:
model_dir = "/home/cwinkler/oreilly/models/DeepSeek-R1-0528-Qwen3-8B-4.25bpw-exl2"
config = ExLlamaV2Config(model_dir)
config.arch_compat_overrides()
model = ExLlamaV2(config)
cache = ExLlamaV2Cache(model, max_seq_len = total_cache_tokens, lazy = True)
model.load_autosplit(cache, progress = True)

In [ ]:
tokenizer = ExLlamaV2Tokenizer(config)

In [ ]:
max_new_tokens = 2500

In [ ]:
generator = ExLlamaV2StreamingGenerator(model, cache, tokenizer)

In [ ]:
settings = ExLlamaV2Sampler.Settings.greedy()

In [ ]:
prompt = "<｜begin▁of▁sentence｜><｜User｜>How many 'r's are in 'strawberry'?<｜Assistant｜>"

In [ ]:
input_ids = tokenizer.encode(prompt)
prompt_tokens = input_ids.shape[-1]

In [ ]:
generator.warmup()

In [ ]:
generator.set_stop_conditions([tokenizer.eos_token_id])
generator.begin_stream(input_ids, settings)

In [ ]:
import time

start = time.time()

generated_tokens = 0
while True:
    chunk, eos, _ = generator.stream()
    generated_tokens += 1
    print (chunk, end = "")
    sys.stdout.flush()
    if eos or generated_tokens == max_new_tokens: break

used = time.time() - start
tps = generated_tokens / used
print(f"\n\n{used} seconds, {tps} tokens/s")

In [ ]:
!nvidia-smi